<a href="https://colab.research.google.com/github/sakshamkumar1/tensorflow-notebooks/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-13e1c833-383f-efa5-55ba-a701ac405ac2)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-05-16 14:27:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-05-16 14:27:42 (82.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2023-05-16 14:27:45--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   640KB/s    in 0.9s    

2023-05-16 14:27:46 (640 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
An indepth look at the new world of work and how young people businesses and economies are coping with huge upheaval http://t.co/aYP6zVHm2A

---

Target: 0 (not real disaster)
Text:
She says that she'd love to come help but
The sea would....
Electrocute us all... ??????????

---

Target: 0 (not real disaster)
Text:
@mrsbinker @EmilioRivera48 @davidlabrava Mine are Diesel and Twister both small for the breed but very strong! I have a beautiful pug too

---

Target: 0 (not real disaster)
Text:
RSS: Judge orders Texas to recognize spouse on same-sex death certificate  http://t.co/TZIolfTe5i

---

Target: 1 (real disaster)
Text:
@APhiABeta1907 w/ugliness due 2 your 'ugly'@AMESocialAction Frat's BIOTERRORISMI'm she who's @FBI ID U $tolewant'g another in my Home.@ABC

---



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_df.iloc[:2].to_numpy()

array([[1, nan, nan,
        'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
        1],
       [4, nan, nan, 'Forest fire near La Ronge Sask. Canada', 1]],
      dtype=object)

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [13]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [17]:
len(train_sentences[0].split())

7

In [18]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
text_vectorizer.adapt(train_sentences)

In [20]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Fatality  https://t.co/407V1y4HHg        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[626,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
text_vectorizer(random_sentence)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([626,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab:{bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [24]:
len(words_in_vocab)

10000

### Creating an Embedding using an Embedding Layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)
embedding

In [26]:
random_sentence = random.choice(train_sentences)
print(f"Original test:\n{random_sentence}\
        \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original test:
@KapoKekito on northgate by the taco truck that's fire.        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01820775, -0.00893121,  0.02427738, ...,  0.04609409,
         -0.01858692,  0.03082912],
        [-0.04061793, -0.00854253,  0.04992861, ...,  0.04167633,
          0.01501891, -0.03272346],
        [ 0.01820775, -0.00893121,  0.02427738, ...,  0.04609409,
         -0.01858692,  0.03082912],
        ...,
        [-0.02916613, -0.00299014,  0.00744433, ..., -0.04006977,
          0.00804549, -0.03578651],
        [-0.02916613, -0.00299014,  0.00744433, ..., -0.04006977,
          0.00804549, -0.03578651],
        [-0.02916613, -0.00299014,  0.00744433, ..., -0.04006977,
          0.00804549, -0.03578651]]], dtype=float32)>

In [27]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 1.82077549e-02, -8.93120840e-03,  2.42773779e-02,  3.61290835e-02,
       -4.45358455e-05, -4.70777154e-02,  3.47287543e-02,  1.74379386e-02,
       -2.35663187e-02,  2.24614479e-02, -4.77882884e-02, -2.12076660e-02,
       -1.16077065e-02,  3.44069861e-02,  3.65641229e-02,  1.66379549e-02,
        4.32889350e-02, -2.38954667e-02, -3.41613442e-02, -4.53332439e-02,
       -1.29022375e-02, -3.71407345e-03, -1.46002285e-02,  4.48830388e-02,
        2.49441527e-02, -3.04604899e-02,  3.27267908e-02, -1.10001788e-02,
        4.82083894e-02, -1.22892037e-02, -4.41459082e-02, -4.03825045e-02,
       -1.03202574e-02,  1.39262341e-02, -4.12805453e-02, -4.40016389e-02,
        4.74250317e-03,  3.41672339e-02, -3.55899334e-03, -4.73196767e-02,
       -5.66570833e-03, -7.30482489e-03,  3.01972516e-02,  1.77095085e-03,
        3.71597670e-02, -1.01903565e-02,  1.78936385e-02, -3.48504670e-02,
        4.04010154e-02, -3.40397283e-03, -3.94534953

### Model 0: Getting a Baseline

In [28]:
# from sklearn.naive_bayes import GaussianNB

# gnb = GaussianNB()
# gnb.fit(train_sentences, train_labels)
# gnb.score(val_sentences, val_labels)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [31]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating an evaluation function for our model experiments

In [32]:
# Function to evaluate: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [33]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model (feed forward neural network)

In [34]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [35]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [36]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [37]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230516-142751
Epoch 1/5
215/215 [==============================] - 30s 114ms/step - loss: 0.6127 - accuracy: 0.6926 - val_loss: 0.5358 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.4437 - accuracy: 0.8178 - val_loss: 0.4707 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3498 - accuracy: 0.8599 - val_loss: 0.4670 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2871 - accuracy: 0.8910 - val_loss: 0.4656 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2406 - accuracy: 0.9110 - val_loss: 0.4878 - val_accuracy: 0.7900


In [38]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4878 - accuracy: 0.7900


[0.48775923252105713, 0.7900262475013733]

In [39]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.3293347 ],
       [0.71019685],
       [0.9975612 ],
       [0.09598737],
       [0.11956416],
       [0.9232108 ],
       [0.90534264],
       [0.99138266],
       [0.95776284],
       [0.22899432]], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [41]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [42]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.7986440245389494,
 'recall': 0.7900262467191601,
 'f1': 0.7859832179841351}

In [43]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Model 2: LSTM

In [45]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.LSTM(64, activation="tanh", return_sequences=True)(x)
print(f"After LSTM cell: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 15, 64)


In [46]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230516-142831
Epoch 1/5
215/215 [==============================] - 17s 54ms/step - loss: 0.2206 - accuracy: 0.9232 - val_loss: 0.5860 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1586 - accuracy: 0.9394 - val_loss: 0.5890 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1321 - accuracy: 0.9511 - val_loss: 0.7954 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1077 - accuracy: 0.9597 - val_loss: 0.7034 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0828 - accuracy: 0.9664 - val_loss: 0.9792 - val_accuracy: 0.7769


In [47]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


((762, 1),
 array([[3.6859728e-02],
        [9.3584847e-01],
        [9.9995041e-01],
        [1.3728189e-01],
        [2.0961250e-04],
        [9.9950802e-01],
        [9.2792284e-01],
        [9.9995434e-01],
        [9.9994540e-01],
        [8.3433306e-01]], dtype=float32))

In [48]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [49]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.69028871391076,
 'precision': 0.7773989686400665,
 'recall': 0.7769028871391076,
 'f1': 0.7755736342533732}

## Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.GRU(64, activation="tanh", return_sequences=True)(x)
print(f"After GRU cell: {x.shape}")
x = layers.GRU(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

After embedding: (None, 15, 128)
After GRU cell: (None, 15, 64)


In [51]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230516-142917
Epoch 1/5
215/215 [==============================] - 17s 61ms/step - loss: 0.1495 - accuracy: 0.9439 - val_loss: 0.6428 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0885 - accuracy: 0.9686 - val_loss: 0.7629 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0708 - accuracy: 0.9718 - val_loss: 1.0717 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 12ms/step - loss: 0.0607 - accuracy: 0.9740 - val_loss: 1.1584 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0513 - accuracy: 0.9777 - val_loss: 1.3431 - val_accuracy: 0.7625


## Model 4: Bidirectional RNN Model

In [52]:
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.Bidirectional(LSTM(64, activation="tanh", return_sequences=True))(x)
print(f"After Bidirectional LSTM cell: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional_LSTM")

After embedding: (None, 15, 128)
After Bidirectional LSTM cell: (None, 15, 128)
(None, 64)


In [53]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230516-143001
Epoch 1/5
215/215 [==============================] - 19s 63ms/step - loss: 0.0985 - accuracy: 0.9693 - val_loss: 1.1567 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0478 - accuracy: 0.9780 - val_loss: 1.2663 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0488 - accuracy: 0.9778 - val_loss: 1.5006 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0431 - accuracy: 0.9788 - val_loss: 1.4025 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0405 - accuracy: 0.9810 - val_loss: 1.4837 - val_accuracy: 0.7651


In [54]:
model_4.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 1.4837 - accuracy: 0.7651


[1.4836702346801758, 0.7650918364524841]

In [55]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10], val_labels[:10]

24/24 [==============================] - 1s 5ms/step


(array([[6.0265544e-03],
        [7.0527589e-01],
        [9.9989748e-01],
        [6.8142541e-02],
        [8.6486769e-05],
        [9.9982965e-01],
        [9.7868323e-01],
        [9.9993062e-01],
        [9.9991238e-01],
        [7.9876590e-01]], dtype=float32),
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0]))

In [56]:
model_4_pred_probs.shape

(762, 1)

In [57]:
val_sentences.shape

(762,)

In [58]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [59]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 76.50918635170603,
 'precision': 0.7675147482403992,
 'recall': 0.7650918635170604,
 'f1': 0.7624764882687336}

## Model 5: Conv1D

In [60]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d_layer = layers.Conv1D(filters=32,
                              kernel_size=5,
                              activation="relu")
conv_1d_output = conv_1d_layer(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
print(f"Embedding output shape: {embedding_test.shape}")
print(f"Conv1D output shape: {conv_1d_output.shape}")
print(f"Max pool  output shape: {max_pool_output.shape}")

Embedding output shape: (1, 15, 128)
Conv1D output shape: (1, 11, 32)
Max pool  output shape: (1, 32)


In [61]:
print(f"Embedding output: {embedding_test}")
print(f"Conv1D output: {conv_1d_output}")
print(f"Max pool output: {max_pool_output}")

Embedding output: [[[ 0.06109811  0.01229846  0.00236952 ... -0.00220561 -0.01761726
   -0.02717866]
  [ 0.04873741  0.01987224  0.00070439 ...  0.00052607  0.04349463
    0.0452583 ]
  [ 0.09167728  0.05433244 -0.03504889 ... -0.02891318 -0.0514228
    0.05787853]
  ...
  [ 0.02253472  0.01374369  0.02565936 ... -0.03087253 -0.00832728
   -0.00614251]
  [ 0.02253472  0.01374369  0.02565936 ... -0.03087253 -0.00832728
   -0.00614251]
  [ 0.02253472  0.01374369  0.02565936 ... -0.03087253 -0.00832728
   -0.00614251]]]
Conv1D output: [[[0.01610602 0.08630361 0.         0.         0.         0.09446689
   0.05456439 0.         0.00560795 0.         0.         0.
   0.06352267 0.         0.02191982 0.         0.         0.07086251
   0.         0.         0.         0.04063533 0.03271006 0.
   0.         0.07214621 0.         0.02233312 0.04464848 0.
   0.02513121 0.        ]
  [0.         0.         0.0075739  0.03388822 0.00588866 0.
   0.01364875 0.         0.0063921  0.         0.     

In [62]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d")

model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 output_layer (Dense)        (None, 1)              

In [63]:
BATCH_SIZE = 32

In [64]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_model_5 = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              batch_size=BATCH_SIZE,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_5_conv1d")])

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230516-143038
Epoch 1/5
215/215 [==============================] - 14s 54ms/step - loss: 0.1408 - accuracy: 0.9545 - val_loss: 0.8533 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0801 - accuracy: 0.9710 - val_loss: 0.9556 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0632 - accuracy: 0.9766 - val_loss: 1.0989 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0568 - accuracy: 0.9772 - val_loss: 1.1669 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0510 - accuracy: 0.9785 - val_loss: 1.1791 - val_accuracy: 0.7664


In [65]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5], model_5_pred_probs.shape

24/24 [==============================] - 0s 4ms/step


(array([[7.6933920e-01],
        [5.1958054e-01],
        [9.9988043e-01],
        [3.6843702e-02],
        [2.9677349e-07]], dtype=float32),
 (762, 1))

In [66]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [67]:
model_5_results = calculate_results(y_true=val_labels,
                                  y_pred=model_5_preds)
model_5_results

{'accuracy': 76.64041994750657,
 'precision': 0.7676724239336781,
 'recall': 0.7664041994750657,
 'f1': 0.7644156046177798}

Model 6: TensorFlow Hub Pretrained Sentence Encoder

In [68]:
sample_sentence

"There's a flood in my street!"

In [69]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                       "When you can the universal sentence encoder on a sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [70]:
embed_samples[1][:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 0.02992674, -0.07772715, -0.01693198,  0.01940703,  0.00121213,
       -0.05806176, -0.07014291,  0.01900714,  0.05421992,  0.05631085,
       -0.04905614, -0.02482399,  0.03954631,  0.08048374,  0.07105057,
       -0.06532165,  0.0129113 , -0.05589466, -0.00210039,  0.0367513 ,
        0.00373515,  0.04555811,  0.05498755,  0.04859942, -0.07956811,
        0.07509258, -0.02012385, -0.01468535, -0.0160961 ,  0.05054294,
        0.04069993,  0.04472741, -0.04459134, -0.00769065, -0.08697076,
       -0.02373411,  0.05550093,  0.0690422 , -0.00191015, -0.00593149,
        0.00569243, -0.02402765,  0.06657269, -0.03563758, -0.00641891,
       -0.00722762, -0.0200778 , -0.0535391 , -0.04398567,  0.00176769],
      dtype=float32)>

In [71]:
embed_samples[0].shape

TensorShape([512])

In [72]:
embed_samples[1].shape

TensorShape([512])

In [73]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [74]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid", name="output_layer")
], name="model_6_USE")

model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [75]:
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230516-151703
Epoch 1/5
215/215 [==============================] - 6s 15ms/step - loss: 0.5031 - accuracy: 0.7911 - val_loss: 0.4507 - val_accuracy: 0.7927
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4144 - accuracy: 0.8159 - val_loss: 0.4411 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4004 - accuracy: 0.8221 - val_loss: 0.4321 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3916 - accuracy: 0.8259 - val_loss: 0.4271 - val_accuracy: 0.8136
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3847 - accuracy: 0.8291 - val_loss: 0.4303 - val_accuracy: 0.8084


In [76]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 17ms/step


array([[0.13660029],
       [0.71273977],
       [0.9889242 ],
       [0.19349492],
       [0.743938  ],
       [0.6577317 ],
       [0.98113054],
       [0.97482514],
       [0.9368005 ],
       [0.07848747]], dtype=float32)

In [77]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [78]:
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 80.83989501312337,
 'precision': 0.8116885217223339,
 'recall': 0.8083989501312336,
 'f1': 0.8064396595798841}

## Model 7: TF Hub Pretrained USE but with 10% of training data

In [79]:
train_10_percent_split = int(0.1 * len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]

In [80]:
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [81]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64